In [59]:
import polars as pl
import hvplot.polars

In [3]:
df_news = pl.read_csv("financial_sentiment_analysis.csv")
df_news = df_news.unique()

In [4]:
df = pl.read_csv("data/raw/All_external.csv")

In [ ]:
df = pl.read_csv('data/processed/final_news_data.csv')
df = df.unique()

In [17]:
df

ITI(13D),ITI(impatient),ITI(patient),ITI(insider),ITI(short),date,permco,ret,prc,vol,on_rdq,vol_missing_flag,comnam,Article_title
f64,f64,f64,f64,f64,str,i64,f64,f64,f64,i64,i64,str,str
0.4724077,0.393814,0.071948,0.383181,0.202257,"""2017-07-24""",54685,0.0164,24.17,107754.0,0,0,"""XENCOR INC""",null
0.039688,0.2751487,0.036716,0.42413,0.358818,"""2013-11-18""",45380,-0.002921,68.26,236800.0,0,0,"""ENERSYS""",null
0.195981,0.453489,0.2236682,0.5433246,0.405519,"""2009-07-21""",30446,-0.020127,9.25,200200.0,0,0,"""GREENBRIER COMPANIES INC""",null
0.4830646,0.546527,0.458572,0.527647,0.5676849,"""2016-10-31""",7518,0.022425,26.9,214784.0,0,0,"""INSTEEL INDUSTRIES INC""",null
0.22784,0.344261,0.123093,0.6648823,0.3857122,"""2019-07-12""",54857,-0.022688,5.6,163768.0,0,0,"""ALDEYRA THERAPEUTICS INC""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.398655,0.335359,0.195551,0.379811,0.385379,"""2014-08-27""",2706,-0.009521,78.02,1.211134e6,0,0,"""RANGE RESOURCES CORP""",null
0.172718,0.217842,0.189273,0.259142,0.322175,"""2018-05-22""",8048,-0.001025,97.5,1.5426587e7,0,0,"""MICROSOFT CORP""",null
0.3820056,0.423608,0.154089,0.856056,0.436012,"""2015-09-24""",29811,-0.022951,11.92,51949.0,0,0,"""MIDSOUTH BANCORP INC""",null


In [22]:
df_articles = df.filter(pl.col('Article_title').is_not_null()).select('date', 'comnam', 'Article_title')

In [30]:
news_df = df_articles.join(df_news, left_on=['date', 'comnam', 'Article_title'], right_on=['date', 'comnam', 'Headline'], how='inner').unique()

In [36]:
final_df = df.join(news_df, on=['date', 'comnam', 'Article_title'], how='left').unique(subset=['date', 'comnam', 'Article_title'])

In [106]:
filter_df = final_df.with_columns(
    pl.col("ret").shift(-1).alias("next_day_ret")
).select(['ITI(13D)', 'next_day_ret', 'permco', 'Positive', 'Negative', 'Neutral', 'date']).sort(['permco', 'date']).drop_nulls(subset=['ITI(13D)', 'next_day_ret'])

In [107]:
filter_df = filter_df.with_columns(pl.col('Positive').fill_null(0), pl.col('Negative').fill_null(0), pl.col('Neutral').fill_null(0))

In [108]:
filter_df = filter_df.with_columns(
    (
        (pl.col('Positive') - pl.col('Negative')) 
    ).alias('sentiment_score')
)


ITI(13D),next_day_ret,permco,Positive,Negative,Neutral,date,sentiment_score
f64,f64,i64,f64,f64,f64,str,f64
0.152583,-0.01768,7,0.0,0.0,0.0,"""2009-05-27""",0.0
0.261909,0.025538,7,0.0,0.0,0.0,"""2009-05-28""",0.0
0.096691,-0.010364,7,0.0,0.0,0.0,"""2009-05-29""",0.0
0.089552,0.019651,7,0.0,0.0,0.0,"""2009-06-01""",0.0
0.018411,0.00791,7,0.0,0.0,0.0,"""2009-06-02""",0.0
…,…,…,…,…,…,…,…
0.319228,0.005982,58620,0.0,0.0,0.0,"""2009-08-25""",0.0
0.114453,0.026149,58620,0.0,0.0,0.0,"""2009-08-26""",0.0
0.215733,-0.001274,58620,0.0,0.0,0.0,"""2009-08-27""",0.0
